# Example 3: Simulating an Electric Vehicle (ARX)

This example follows the previous example, but where we instead assume that each cell in the battery pack are modeled as an Autoregressive with Exogenous input (ARX) model. The model is assmued to follow the structure:

$$V_q[k] = a_1V_q[k-1] + \cdots + a_n V_q[k-n] + b_0I[k] + b_1I[k-1] + \cdots + b_n I[k-n] = \boldsymbol{\phi}_k^T\boldsymbol{\theta}_k$$
$$\boldsymbol{\phi}_k = \begin{bmatrix} V_q[k-1] & \cdots & V_q[k-n] & I[k] & I[k-1] & \cdots & I[k-n] \end{bmatrix}^T$$
$$\boldsymbol{\theta}_k = \begin{bmatrix} a_1 & \cdots & a_n & b_0 & b_1 & \cdots & b_n \end{bmatrix}^T$$

where $V_q = V_t - OCV$ and $n$ is the model order.

In [ ]:
from tracksim.tracksim import Vehicle, Pack
from tracksim.vehicle_models import ChevyVoltTuned
from tracksim.pack_models import ChevyVoltPack
from tracksim.cell_models import ARX1 # first order ARX with dynamic parameters

print(ARX1)

Since the nominal capacity of this cell is relatively small, we can increase the dimensions of the battery pack to raise the nominal energy capacity.

In [ ]:
pack_model = ChevyVoltPack.copy()

pack_model['No. Cells Series'] = 36
pack_model['No. Cells Parallel'] = 12

The setup and simulation of the vehicle is the same as before.

In [ ]:
pack = Pack(pack_model, ARX1)
vehicle = Vehicle(ChevyVoltTuned, pack)

In [ ]:
import pandas as pd
from src.tracksim.utils import exp_average # ONLY FOR PRIVATE TESTING
# from tracksim.utils import exp_average

# Get trip data

trip_file = 'simulated_trip_files/veh0.csv'
trip_data = pd.read_csv(trip_file)

time = trip_data['Time [s]']
time_delta = time[1] - time[0]
speed = exp_average(trip_data['Speed [m/s]'], 0.2) # Low-pass filter to make a more realistic speed profile

# Set initial battery conditions

soc_init = 0.8 # Initial State Of Charge (SOC)

vehicle.simulate_vehicle(time, speed, time_delta)
pack.set_initial_conditions(soc=soc_init)
vehicle.simulate_battery_pack()

We can then retrieve the same vehicle, pack, and cell measurements as in the previous example.

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-notebook')

fig, ax = plt.subplots(nrows=3)
ax[0].plot(vehicle.simulation_results['speed_desired']*3.6)
ax[0].set_ylabel('Speed [km/h]')
ax[1].plot(vehicle.simulation_results['acceleration_desired'], label='desired')
ax[1].set_ylabel('Acceleraton [m/s^2]')
ax[1].plot(vehicle.simulation_results['acceleration_actual'], linestyle='--', label='actual')
ax[1].legend()
ax[2].plot(vehicle.simulation_results['battery_demand'])
ax[2].set_ylabel('Battery Power Demand [kW]')
ax[2].set_xlabel('Time [s]')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(nrows=3)
ax[0].plot(pack.simulation_results['pack']['current'])
ax[0].set_ylabel('Pack Current [A]')
ax[1].plot(pack.simulation_results['pack']['voltage'])
ax[1].set_ylabel('Pack Voltage [V]')
ax[2].plot(pack.simulation_results['pack']['avg_soc']*100)
ax[2].set_ylabel('Pack SOC (%)')
ax[2].set_xlabel('Time [s]')

fig.tight_layout()

In [ ]:
cell_i, cell_j = (0,0) # Cell index

fig, ax = plt.subplots(nrows=3)
ax[0].plot(pack.simulation_results[f'cell_{cell_i}-{cell_j}']['current'])
ax[0].set_ylabel('Cell Current [A]')
ax[1].plot(pack.simulation_results[f'cell_{cell_i}-{cell_j}']['voltage'])
ax[1].set_ylabel('Cell Voltage [V]')
ax[2].plot(pack.simulation_results[f'cell_{cell_i}-{cell_j}']['soc']*100)
ax[2].set_ylabel('Cell SOC (%)')
ax[2].set_xlabel('Time [s]')

fig.tight_layout()

We can also extract the time-varying ARX parameters. Since the cell model is a first order ARX, the available parameters are $a_1$, $b_0$, and $b_1$.

In [ ]:
a1 = vehicle.pack.simulation_results['cell_0-0']['a1']
b0 = vehicle.pack.simulation_results['cell_0-0']['b0']
b1 = vehicle.pack.simulation_results['cell_0-0']['b1']

fig, ax = plt.subplots(nrows=3)
ax[0].plot(a1)
ax[0].set_ylabel(r'$a_1$')
ax[1].plot(b0)
ax[1].set_ylabel(r'$b_0$')
ax[2].plot(b1)
ax[2].set_ylabel('$b_1$')
ax[2].set_xlabel('Time [s]')

fig.tight_layout()